# Import libraries and environment keys

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# Helper functions
Used to load images as base64 strings

In [12]:
import base64
from pathlib import Path
import mimetypes

def load_image_as_base64(path: str) -> tuple[str, str]:
    """Return (base64_data, mime_type) for an image file."""
    image_bytes = Path(path).read_bytes()
    mime = mimetypes.guess_type(path)[0] or "image/png"
    return base64.b64encode(image_bytes).decode("utf-8"), mime

def load_image_from_path(path: str) -> bytes:
    """Return image bytes for an image file."""
    with open(path, "rb") as image_file:
        return image_file.read()


# Native Client Integrations

## Amazon and Anthropic Models




### Amazon Bedrock Models - text only

In [ ]:
from gen_ai_hub.proxy.native.amazon.clients import Session

# Model parameters
temperature = 0.6
max_Tokens = 1000
model = "anthropic--claude-4-sonnet"

# Create a session with the model
bedrock = Session().client(model_name=model)
messages = [
    {
        "role": "user",
        "content": [
            {
                "text": "What is the capital of France?"
            }
        ],
    }
]
response = bedrock.converse(
    messages=messages,
    inferenceConfig={"maxTokens": max_Tokens, "temperature": temperature},
)
print(response)


{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 14 Mar 2025 10:40:57 GMT', 'content-type': 'application/json', 'content-length': '213'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': 'The capital of France is Paris.'}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 14, 'outputTokens': 10, 'totalTokens': 24}, 'metrics': {'latencyMs': 663}}


In [4]:
print(response['output']['message']['content'][0]['text'])

The capital of France is Paris.


### Amazon Bedrock Models - text and images

In [13]:
from gen_ai_hub.proxy.native.amazon.clients import Session

# Model parameters
temperature = 0.6
max_Tokens = 1000
model = "anthropic--claude-4-sonnet"

# Create a session with the model
bedrock = Session().client(model_name=model)

# Load image and convert to base64
image_path = "SAP_logo.png"
fmt = "png"  # Format of the image
image_data = load_image_from_path(image_path)


messages = [
    {
        "role": "user",
        "content": [
            {
                "text": "What is the content of the image?"
            },
            {
                "image": {
                    "format": fmt, "source":{"bytes": image_data}
                }
            }
        ]
    }
]

response = bedrock.converse(
    messages=messages,
    inferenceConfig={"maxTokens": max_Tokens, "temperature": temperature},
)
print(response)

{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 20 Oct 2025 11:51:11 GMT', 'content-type': 'application/json', 'content-length': '738', 'x-aicore-request-id': '34e22184-090c-9a57-b441-700f4fb5d2ef', 'x-upstream-service-time': '4480'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': 'This image shows the SAP logo on a grid background. The logo consists of the letters "SAP" in large white text displayed on a blue geometric shape. The background appears to be graph paper or a grid pattern, suggesting this might be from a design or presentation context. SAP is a well-known German multinational software corporation that makes enterprise software to manage business operations and customer relations.'}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 1194, 'outputTokens': 83, 'totalTokens': 1277, 'cacheReadInputTokens': 0, 'cacheWriteInputTokens': 0}, 'metrics': {'latencyMs': 4283}}


In [ ]:
print(response['output']['message']['content'][0]['text'])

This image shows the SAP logo on a grid background. The logo consists of the letters "SAP" in large white text displayed on a blue geometric shape. The background appears to be graph paper or a grid pattern, suggesting this might be from a design or presentation context. SAP is a well-known German multinational software corporation that makes enterprise software to manage business operations and customer relations.


## OpenAI Models



### OpenAI Models - text only

In [2]:
from gen_ai_hub.proxy.native.openai import chat

# Model parameters
temperature = 0.6
max_Tokens = 1000
model = "gpt-4o"  # Also compatible with Meta models like meta-llama3.1-70b-instruct
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"}
]

# Create a session with the model
response = chat.completions.create(messages=messages, model=model, temperature=temperature, max_tokens=max_Tokens)
print(response)


ChatCompletion(id='chatcmpl-CSiMjjjEgRVXLVhZVr692UerP2avs', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of France is Paris.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1760959761, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_ee1d74bde0', usage=CompletionUsage(completion_tokens=8, prompt_tokens=24, total_tokens=32, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)), prompt_filter_results=[{'content_filter_res

In [ ]:
print(response.choices[0].message.content)
usage = getattr(response, "usage", None)
if usage:
    print(f"Prompt tokens: {usage.prompt_tokens}")
    print(f"Completion tokens: {usage.completion_tokens}")
    print(f"Total tokens: {usage.total_tokens}")

The capital of France is Paris.
Prompt tokens: 24
Completion tokens: 8
Total tokens: 32


### OpenAI Models - text and images

In [ ]:
model = "gpt-4o"
image_path = "SAP_logo.png"
base64_data, mime_type = load_image_as_base64(image_path)
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that can answer questions and help with tasks."
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "What is the content of the image?"
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:{mime_type};base64,{base64_data}"
                }
            }
        ]
    }
]


response = chat.completions.create(
    messages=messages,
    model=model,
    temperature=temperature,
)

print(response.choices[0].message.content)
usage = getattr(response, "usage", None)
if usage:
    print(f"Prompt tokens: {usage.prompt_tokens}")
    

The image contains the logo of SAP, which is a multinational software corporation that makes enterprise software to manage business operations and customer relations. The logo features the letters "SAP" in bold white font on a blue background with a triangular shape extending from the top right corner.
Prompt tokens: 1138


### OpenAI Models - Reasoning models
For reasoning tasks, we can define the "thinking budget"

In [ ]:
# Model parameters
model = "gpt-5" 
messages = [
    {"role": "user", "content": "What is Green theorem?"}
]

# Create a session with the model
import time

# List of reasoning efforts
reasoning_efforts = ["minimal", "low", "medium", "high"]
responses = {}

# Time the response for each reasoning effort
for effort in reasoning_efforts:
    start_time = time.time()
    response = chat.completions.create(messages=messages, model=model, reasoning_effort=effort)  # OpenAI reasoning models do not allow temperature or max_tokens settings
    end_time = time.time()
    elapsed = end_time - start_time
    responses[effort] = {
        "response": response,
        "time_seconds": elapsed
    }
    print(f"Reasoning effort: {effort}, Time taken: {elapsed:.3f} seconds")
print(response.choices[0].message.content[:300] + '...')


Reasoning effort: minimal, Time taken: 3.657 seconds
Reasoning effort: low, Time taken: 8.252 seconds
Reasoning effort: medium, Time taken: 26.256 seconds
Reasoning effort: high, Time taken: 21.703 seconds
Green’s theorem is a fundamental result in planar vector calculus that relates a line integral aroun


# Google Vertex AI Models

### Google Vertex AI Models - text only

In [ ]:
from gen_ai_hub.proxy.native.google_vertexai.clients import GenerativeModel

model_name = "gemini-2.5-pro"

chat = GenerativeModel(model_name)
prompt = "What is the capital of France?"
generation_config = {"temperature": 0.}

response = chat.generate_content(contents=prompt, generation_config=generation_config)
print(response.text)




/Users/I760054/Documents/programs/Best Practices/sap-btp-ai-best-practices/.venv/lib/python3.10/site-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


The capital of France is **Paris**.


### Google Vertex AI Models - text and images

In [24]:
from gen_ai_hub.proxy.native.google_vertexai.clients import GenerativeModel

model_name = "gemini-2.5-pro"
image_path = "SAP_logo.png"
base64_data, mime_type = load_image_as_base64(image_path)

chat = GenerativeModel(model_name)
contents = [
    {
        "role": "user",
        "parts": [
            {"text": "What is the content of the image?"},
            {"inline_data": {"mime_type": mime_type, "data": base64_data}}
        ] 
    }
]
generation_config = {"temperature": 0.}

response = chat.generate_content(contents=contents, generation_config=generation_config)
print(response.text)



This image displays the logo of the company **SAP**.

The logo features the letters "SAP" in a large, bold, white, sans-serif typeface. These letters are placed on a blue shape that has a gradient, transitioning from a darker blue at the bottom to a lighter, brighter blue at the top. The right side of this blue shape is cut off with a diagonal line sloping downwards.

The entire logo is set against a light gray background with a white grid pattern, similar to graph paper, which often suggests precision and design. There are also ruler-like markings along the top and bottom edges of the image.
